- TODO: Parallelize sharp wave property computation

# Imports and definitions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import json
from datetime import datetime
from ast import literal_eval

In [4]:
from ecephys_analyses.data import paths, channel_groups
from ecephys.sglx_utils import load_timeseries, load_multifile_timeseries
from ecephys.signal.csd import get_kcsd
from ecephys.signal.sharp_wave_ripples import detect_sharp_waves_by_value, detect_sharp_waves_by_zscore, get_durations, get_midpoints, get_sink_amplitudes, get_sink_integrals
from ecephys.utils import load_df_h5, store_df_h5, zscore_to_value
import ecephys.plot as eplt

## Old, static channel based method

In [5]:
def get_spw_detection_parameters(subject, detection_threshold_zscore=2.5, boundary_threshold_zscore=1):
    sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]
    hpc_chans = channel_groups.hippocampus[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition="recovery-sleep-2h", ext="lf.bin")
    params_path = paths.get_datapath(subject=subject, condition="sleep-homeostasis", file="sharp_wave_detection_params.json")
    
    if len(bin_paths) > 1:
        (time, hpc_lfps, fs) = load_multifile_timeseries(bin_paths, hpc_chans)
    else:
        (time, hpc_lfps, fs) = load_timeseries(bin_paths[0], hpc_chans)
    
    gdx = intersite_distance = 0.020
    k = get_kcsd(
        hpc_lfps, intersite_distance=intersite_distance, gdx=gdx, do_lcurve=True
    )
    
    hpc_csd = k.values("CSD")
    sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]
    combined_csd = np.sum(-sr_csd.T, axis=1)

    detection_threshold = zscore_to_value(combined_csd, detection_threshold_zscore)
    boundary_threshold = zscore_to_value(combined_csd, boundary_threshold_zscore)
    
    metadata = dict(
        csd_chans=hpc_chans.tolist(),
        detection_chans=sr_chans,
        electrode_positions=k.ele_pos.tolist(),
        intersite_distance=intersite_distance,
        gdx=k.gdx,
        lambd=k.lambd,
        R=k.R,
        detect_states=["all"],
        detection_threshold_zscore=detection_threshold_zscore,
        boundary_threshold_zscore=boundary_threshold_zscore,
        detection_threshold=detection_threshold,
        boundary_threshold=boundary_threshold,
        minimum_duration=0.005,
        params_source_files=[str(path) for path in bin_paths] 
    )
    
    params_path.parent.mkdir(parents=True, exist_ok=True)
    with open(params_path, "x") as params_file:
        json.dump(metadata, params_file, indent=4)

In [6]:
def run_spw_detection_pipeline_on_file(
    bin_path, spw_path, params_path, sr_chans, hpc_chans
):
    hpc_lfps = load_timeseries(bin_path, hpc_chans)
    (time, fs) = (hpc_lfps.time.values, hpc_lfps.fs)

    with open(params_path) as params_file:
        params = json.load(params_file)

    intersite_distance = params["intersite_distance"]
    k = get_kcsd(
        hpc_lfps.values,
        intersite_distance=params["intersite_distance"],
        gdx=params["gdx"],
        lambd=params["lambd"],
        R_init=params["R"],
        do_lcurve=False,
    )

    hpc_csd = k.values("CSD")
    sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

    spws = detect_sharp_waves_by_value(
        time,
        sr_csd,
        params["detection_threshold"],
        params["boundary_threshold"],
        params["minimum_duration"],
    )

    spws["duration"] = get_durations(spws)
    spws["midpoint"] = get_midpoints(spws)
    spws["sink_amplitude"] = get_sink_amplitudes(spws, time, sr_csd) * (
        1e-6
    )  # Scale to mA/mm
    spws["sink_integral"] = (
        get_sink_integrals(spws, time, fs, sr_csd) * (1e-6) * (1e3)
    )  # Scale to mA * ms

    metadata = dict(
        csd_chans=hpc_chans,
        detection_chans=sr_chans,
        electrode_positions=k.ele_pos,
        intersite_distance=intersite_distance,
        gdx=k.gdx,
        lambd=k.lambd,
        R=k.R,
        detect_states=["all"],
        file_start=hpc_lfps.fileCreateTime,
    )
    metadata.update(spws.attrs)

    store_df_h5(spw_path, spws, **metadata)

In [7]:
def run_spw_detection_pipeline_on_condition(subject, condition):
    sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]
    hpc_chans = channel_groups.hippocampus[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="lf.bin")
    spw_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="spws.h5")
    params_path = paths.get_datapath(subject=subject, condition="sleep-homeostasis", file="sharp_wave_detection_params.json")

    for bin_path, spw_path in zip(bin_paths, spw_paths):
        run_spw_detection_pipeline_on_file(bin_path, spw_path, params_path, sr_chans, hpc_chans)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

## New, drift method

In [32]:
def get_epoch_spws(hpc_csd, params, epoch_start, epoch_end, sr_chans):
    sr_csd = hpc_csd.sel(time=slice(epoch_start, epoch_end), channel=sr_chans)
    
    spws = detect_sharp_waves_by_value(
        sr_csd.time.values,
        sr_csd.values,
        params["detection_threshold"],
        params["boundary_threshold"],
        params["minimum_duration"],
    )

    spws["duration"] = get_durations(spws)
    spws["midpoint"] = get_midpoints(spws)
    spws["sink_amplitude"] = get_sink_amplitudes(spws, sr_csd.time.values, sr_csd.values) * (
        1e-6
    )  # Scale to mA/mm
    spws["sink_integral"] = (
        get_sink_integrals(spws, sr_csd.time.values, sr_csd.fs, sr_csd.values) * (1e-6) * (1e3)
    )  # Scale to mA * ms
    
    spws["sr_chans"] = [sr_chans] * len(spws)
    
    return spws

In [33]:
def get_file_spws(bin_path, sr_chans_path, spw_path, params_path, hpc_chans):
    hpc_lfps = load_timeseries(bin_path, hpc_chans)

    with open(params_path) as params_file:
        params = json.load(params_file)

    intersite_distance = params["intersite_distance"]
    k = get_kcsd(
        hpc_lfps.values,
        intersite_distance=params["intersite_distance"],
        gdx=params["gdx"],
        lambd=params["lambd"],
        R_init=params["R"],
        do_lcurve=False,
    )
    
    hpc_csd = xr.DataArray(
        k.values("CSD"),
        dims=("channel", "time"),
        coords={"channel": hpc_lfps.channel.values, "time": hpc_lfps.time.values},
        attrs={'units': "nA/mm", 'fs': hpc_lfps.fs}
    ) 

    spws_by_epoch = list()
    sr_chans_df = pd.read_csv(sr_chans_path, converters={"sr_chans": literal_eval})
    sr_chans_df.sr_chans = sr_chans_df.sr_chans.apply(list)
    for epoch in sr_chans_df.itertuples():
        spws_by_epoch.append(get_epoch_spws(hpc_csd, params, epoch.start_time, epoch.end_time, epoch.sr_chans))
    
    spws = pd.concat(spws_by_epoch)
    metadata = dict(
        csd_chans=hpc_chans,
        electrode_positions=k.ele_pos,
        intersite_distance=intersite_distance,
        gdx=k.gdx,
        lambd=k.lambd,
        R=k.R,
        detect_states=["all"],
        file_start=hpc_lfps.fileCreateTime,
    )
    metadata.update(spws.attrs)

    store_df_h5(spw_path, spws, **metadata)

In [34]:
def get_condition_spws(subject, condition):
    hpc_chans = channel_groups.hippocampus[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="lf.bin")
    sr_chans_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="sr_chans.csv")
    spw_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="spws.h5")
    params_path = paths.get_datapath(subject=subject, condition="sleep-homeostasis", file="sharp_wave_detection_params.json")

    for bin_path, sr_chans_path, spw_path in zip(bin_paths, sr_chans_paths, spw_paths):
        get_file_spws(bin_path, sr_chans_path, spw_path, params_path, hpc_chans)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

# Run automated pipeline

### Segundo

In [16]:
get_spw_detection_parameters(subject="Segundo", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 18000001
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-11
max lambda 0.0133
min lambda 1e-11
max lambda 0.0133
l-curve (all lambda):  0.23
Best lambda and R =  0.00013089207687291224 ,  0.23


In [17]:
run_spw_detection_pipeline_on_condition(subject="Segundo", condition="sleep-deprivation-12h-circadian-match")

nChan: 385, nFileSamp: 18000000
11:26:20: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t8.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
11:35:25: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t9.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
11:51:12: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t10.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
11:59:14: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t11.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
12:06:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t12.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
12:16:12: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_

In [ ]:
run_spw_detection_pipeline_on_condition(subject="Segundo", condition="extended-wake")

In [12]:
run_spw_detection_pipeline_on_condition(subject="Segundo", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000001
18:03:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t23.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
18:19:47: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t24.imec0.lf.bin
nChan: 385, nFileSamp: 18000001
18:34:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/raw/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t25.imec0.lf.bin


### Valentino

In [19]:
get_spw_detection_parameters(subject="Valentino", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 18000000
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0119
min lambda 1e-12
max lambda 0.0119
l-curve (all lambda):  0.23
Best lambda and R =  0.00105886675336374 ,  0.23


In [35]:
get_condition_spws(subject="Valentino", condition="all")

nChan: 385, nFileSamp: 18000000


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.7/site-packages/ipykernel_launcher.py:10: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['sr_chans'], dtype='object')]

  # Remove the CWD from sys.path while we load stuff.


19:14:40: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
19:18:55: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
19:25:07: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t2.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
19:31:21: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
19:36:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
19:43:01: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/

### Doppio

In [ ]:
get_spw_detection_parameters(subject="Doppio", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

In [11]:
run_spw_detection_pipeline_on_condition(subject="Doppio", condition="extended-wake-6h-circadian-match")

nChan: 385, nFileSamp: 18000019
16:05:21: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
16:15:37: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
16:23:25: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t2.imec0.lf.bin


In [12]:
run_spw_detection_pipeline_on_condition(subject="Doppio", condition="recovery-sleep-6h-circadian-match")

nChan: 385, nFileSamp: 18000019
16:34:35: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
16:44:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
16:53:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t5.imec0.lf.bin


In [ ]:
run_spw_detection_pipeline_on_condition(subject="Doppio", condition="extended-wake-6h")

In [10]:
run_spw_detection_pipeline_on_condition(subject="Doppio", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 18000019
17:29:26: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
17:38:31: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
17:47:35: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/raw/3-18-2020_g0/3-18-2020_g0_imec0/3-18-2020_g0_t5.imec0.lf.bin


### Alessandro

In [18]:
get_spw_detection_parameters(subject="Alessandro", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
You are loading multifile SGLX data without xarray.
 Are you sure you want to do this? Please see documentation.
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0126
min lambda 1e-12
max lambda 0.0126
l-curve (all lambda):  0.23
Best lambda and R =  3.711471263028718e-05 ,  0.23


In [ ]:
run_spw_detection_pipeline_on_condition(subject="Alessandro", condition="sleep-deprivation-12h-circadian-match")

nChan: 385, nFileSamp: 9000051
15:54:54: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
15:57:03: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:00:06: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:02:33: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 9000051
16:04:29: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 9000051
16:06:57: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-24-2020_g0/8-24

In [31]:
run_spw_detection_pipeline_on_condition(subject="Alessandro", condition="extended-wake")

nChan: 385, nFileSamp: 9000051
16:17:22: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:20:02: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 8127109
16:22:28: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD_g0/8-25-2020_SD_g0_imec0/8-25-2020_SD_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 8277840
16:24:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_SD2_g0/8-25-2020_SD2_g0_imec0/8-25-2020_SD2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:27:55: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
16:31:12: Finished /Volumes/neuropixel_archive/Data/chronic/

In [13]:
run_spw_detection_pipeline_on_condition(subject="Alessandro", condition="recovery-sleep-6h")

nChan: 385, nFileSamp: 9000052
12:34:40: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:41:25: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t3.imec0.lf.bin
nChan: 385, nFileSamp: 9000051
12:46:54: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t4.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:50:03: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t5.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:53:13: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25-2020_g0_imec0/8-25-2020_g0_t6.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
12:55:58: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/raw/8-25-2020_g0/8-25

### Eugene

In [34]:
get_spw_detection_parameters(subject="Eugene", detection_threshold_zscore=2.5, boundary_threshold_zscore=1)

nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
You are loading multifile SGLX data without xarray.
 Are you sure you want to do this? Please see documentation.
Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-11
max lambda 0.0140
min lambda 1e-11
max lambda 0.0140
l-curve (all lambda):  0.23
Best lambda and R =  0.001404472887665455 ,  0.23


In [8]:
run_spw_detection_pipeline_on_condition(subject="Eugene", condition="sleep-deprivation-12h-circadian-match")

nChan: 385, nFileSamp: 9000025
17:43:47: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_BL_24hs_g0/9.24.2020_BL_24hs_g0_imec0/9.24.2020_BL_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
17:49:02: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
17:53:14: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
17:57:40: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
18:02:12: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t3.imec0.lf.bin
nChan: 385, nFileS

In [35]:
run_spw_detection_pipeline_on_condition(subject="Eugene", condition="extended-wake")

nChan: 385, nFileSamp: 800217
16:55:56: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs_g0/9.24.2020_SD_24hs_g0_imec0/9.24.2020_SD_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 2877424
16:56:28: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs1_g0/9.24.2020_SD_24hs1_g0_imec0/9.24.2020_SD_24hs1_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 2430722
16:56:59: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs2_g0/9.24.2020_SD_24hs2_g0_imec0/9.24.2020_SD_24hs2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 5735230
16:58:16: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs3_g0/9.24.2020_SD_24hs3_g0_imec0/9.24.2020_SD_24hs3_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
17:01:55: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SD_24hs3_g1/9.24.2020_SD_24hs3_g1_imec0/9.24.2020_SD_24hs3_g1_t0.imec0.lf.bin
nChan: 385, nFileSa

In [ ]:
run_spw_detection_pipeline_on_condition(subject="Eugene", condition="recovery-sleep-6h")

In [13]:
run_spw_detection_pipeline_on_condition(subject="Eugene", condition="recovery-sleep-patch")

nChan: 385, nFileSamp: 510366
16:53:33: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs_g0/9.24.2020_SR_24hs_g0_imec0/9.24.2020_SR_24hs_g0_t5.imec0.lf.bin
nChan: 385, nFileSamp: 5341730
16:55:20: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/raw/9.24.2020_SR_24hs1_g0/9.24.2020_SR_24hs1_g0_imec0/9.24.2020_SR_24hs1_g0_t0.imec0.lf.bin


# Run pipeline piecemeal

In [19]:
subject = "Valentino"
condition = "recovery-sleep-2h"

## Load the data

In [21]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]
hpc_chans = channel_groups.hippocampus[subject]
bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition="recovery-sleep-2h", ext="lf.bin")
bin_path = bin_paths[0]
params_path = paths.get_datapath(subject=subject, condition="sleep-homeostasis", file="sharp_wave_detection_params.json")

In [22]:
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[subject]

In [23]:
hpc_lfps = load_timeseries(bin_path, hpc_chans, start_time=0, end_time=600)
(time, fs) = (hpc_lfps.time.values, hpc_lfps.fs)

nChan: 385, nFileSamp: 18000000


In [92]:
#df = pd.DataFrame({'start_time': [0.0, 30.0, 60.0, 100.0, 500.0], 'end_time': [30.0, 60.0, 100.0, 500.0, 600.0], 'sr_chans': [[165, 166, 169, 170], [165, 166, 169, 170], [165, 166, 169, 170], [165, 166, 169, 170], [165, 166, 169, 170]]})

#df.to_csv('/Volumes/neuropixel/Data/foo.csv')
#from ast import literal_eval
#df2 = pd.read_csv('/Volumes/neuropixel/Data/foo.csv', converters={"sr_chans": literal_eval})

#df.sr_chans = df.sr_chans.apply(list)

#row = 0
#hpc_lfps.sel(time=slice(df.start_time[row], df.end_time[row]), channel=df.sr_chans[row])

### Explore LFPs (optional)

In [ ]:
eplt.lfp_explorer(time, hpc_lfps, chan_labels=hpc_chans)

## Detect sharp waves

### If we need to determine detection parameters

In [8]:
intersite_distance = 0.020
k = get_kcsd(hpc_lfps, intersite_distance=intersite_distance, gdx=0.020, do_lcurve=True)
hpc_csd = k.values('CSD')
sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0126
min lambda 1e-12
max lambda 0.0126
l-curve (all lambda):  0.23
Best lambda and R =  0.0003822395851068327 ,  0.23


In [32]:
spws = detect_sharp_waves_by_zscore(time, sr_csd)

### If we are using detection parameters obtained elsewhere

In [24]:
with open(params_path) as params_file:
    params = json.load(params_file)
intersite_distance = params['intersite_distance']

In [25]:
k = get_kcsd(hpc_lfps.values, intersite_distance=params['intersite_distance'], gdx=params['gdx'], lambd=params['lambd'], R_init=params['R'], do_lcurve=False)
hpc_csd = k.values('CSD')
sr_csd = hpc_csd[np.isin(hpc_chans, sr_chans)]

In [29]:
time_mask = np.logical_and(time >= 6, time <= 78)
chan_mask = np.isin(hpc_chans, sr_chans)
#sr_csd = hpc_csd[chan_mask, time_mask]

In [44]:
hpc_csd[chan_mask][:, time_mask]

array([[-17887.42669554, -18414.29974266, -18407.8830503 , ...,
          3689.77681678,   3531.04687352,   3311.70395006],
       [-17447.7442046 , -17867.06168193, -17898.46261739, ...,
          5132.57043659,   4818.28779426,   4509.65873559],
       [-16657.17328571, -16985.48824587, -17078.41235388, ...,
          6265.83833846,   5815.91040485,   5433.05196922],
       [-15475.82032251, -15731.49664498, -15904.60754366, ...,
          7051.9042419 ,   6489.53637675,   6048.8042175 ]])

In [12]:
spws = detect_sharp_waves_by_value(time, sr_csd, params['detection_threshold'], params['boundary_threshold'], params['minimum_duration'])

## Compute SPW properties

In [13]:
spws["duration"] = get_durations(spws)
spws["midpoint"] = get_midpoints(spws)

In [14]:
spws["sink_amplitude"] = get_sink_amplitudes(spws, time, sr_csd) * (1e-6) # Scale to mA/mm

In [15]:
spws["sink_integral"] = get_sink_integrals(spws, time, fs, sr_csd) * (1e-6) * (1e3) # Scale to mA * ms

## Export results

In [16]:
metadata = dict(
    csd_chans=hpc_chans,
    detection_chans=sr_chans,
    electrode_positions=k.ele_pos, 
    intersite_distance=intersite_distance,
    gdx=k.gdx,
    lambd = k.lambd,
    R = k.R,
    detect_states=["all"],
)
metadata.update(spws.attrs)
spws_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="sharp_waves.h5")
store_df_h5(spws_path, spws, **metadata)

## If necessary, create params file

In [85]:
metadata.update({'params_source_file': str(bin_path)})
metadata['csd_chans'] = metadata['csd_chans'].tolist()
metadata['electrode_positions'] = metadata['electrode_positions'].tolist()
with open(params_path, 'x') as params_file:
    json.dump(metadata, params_file, indent=4)

In [23]:
df = pd.DataFrame()

In [24]:
df["foo"] = np.arange(100)

In [26]:
bar = [1, 2, 3]

In [30]:
df["bar"] = [bar] * len(df)

In [31]:
df

,foo,bar
0,0,"[1, 2, 3]"
1,1,"[1, 2, 3]"
2,2,"[1, 2, 3]"
3,3,"[1, 2, 3]"
4,4,"[1, 2, 3]"
...,...,...
95,95,"[1, 2, 3]"
96,96,"[1, 2, 3]"
97,97,"[1, 2, 3]"
98,98,"[1, 2, 3]"
